# Algoritmia
## Práctica Obligatoria 1
### Curso 2021 - 2022
###### Métodos Voraces
---
 

#### Autores:
* Patricia Hernando Fernández

---
Resuelva la siguiente práctica.

Importe las librerías que desees
**Recuerda**: 
* Solamente puedes utilizar bibliotecas nativas (https://docs.python.org/es/3.6/library/index.html)
* Las funciones que importes no son "gratis", cada una tendrá una complejidad temporal y espacial que se tendrá que tener en cuenta.



---
**Notas:**
* Una vez se selecciona un candidato o se acepta o se rechaza. No tener bucles muy complicados que comparen y comparen, ya no sería voraz.
* Podemos sacarlo de internet poniendo referencias o si inventas algo y demuestras que funciona decírselo.
* No liarse en los Hash, no usar librerias ni nada. Con devolver el name+size o algo así vale.

In [1]:
#Importante ≥ estoy utilizando la versión 3.9.7 de Python

from platform import python_version
print(python_version())

3.11.8


In [2]:
# Imports
import heapq

In [3]:
class Video:
    """
    Clase Video. 
    Representa una serie o película.
    """    

    def __init__(self, name, size):
        """Crea un objeto de clase Video

        Parameters
        ----------
        name : str
            Nombre de la serie/película
        size : number
            Tamaño en memoria de la serie/película
        """

        self.name = name
        self.size = size

        #Diccionario. Para un país, guarda el número de usuarios.
        self.users = {}
    
    def __hash__(self):

        """Genera el valor hash identificativo del vídeo

        Returns
        -------
        int
            Valor hash
        """

        return self.name.__hash__() + self.size

    def __str__(self):
        """Genera una cadena descriptiva del objeto

        Returns
        -------
        str
            Cadena descriptiva
        """        
        return "Video: {}, tamaño: {} GB.".format(self.name, self.size)


    def __repr__(self):
        """Genera una cadena descriptiva del objeto dentro de colecciones

        Returns
        -------
        str
            Cadena descriptiva
        """  
        return self.name

    def set_users(self, country, users):
        """Dado un pais y un número de usuarios
           almacena para este vídeo la cantidad de espectadores que tiene.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película
        users : int
            Número de espectadores
        """        

        self.users[country] = users

    def get_users(self, country):
        """Dado un país, obtiene el número de usuarios.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película

        Returns
        -------
        int
            Número de espectadores para el país `country`
        """ 
        
        usuarios = self.users.get(country)
        
        if usuarios is None:
            return 0
        
        else:
            return usuarios
        
    #Métodos propios

    """Compara dos objetos de la clase vídeo.

        Returns
        -------
        bool
            True si el primero es mejor, false si no.
        """

    def __lt__(self, other):
        #En caso de que haya dos vídeos con el mismo número de espectadores, comparo por tamaño.
        return self.size < other.size

In [4]:
class ServidorCache:
    """
    Clase del servidor caché donde se almacenan parte de series/películas.
    """
    
    def __init__(self, identifier, country, capacity):
        """Instancia un Servidor de Caché

        Parameters
        ----------
        identifier : int
            Valor que identifica un servidor.
        country : str
            País donde está el servidor.
        capacity : int
            Cantidad de memoria de almacenamiento disponible.
        """
        self.identifier = identifier
        self.country = country
        self.capacity = capacity

        #Diccionario. Clave (video), valor (cantidad en proporcion).
        self.videos_almacenados =  {}

        #Espacio ocupado en el servidor
        self.espacio_ocupado = 0
        
    def __hash__(self):
        """Genera el valor hash identificativo del servidor

        Returns
        -------
        int
            Valor hash
        """
        return self.identifier * self.country.__hash__()
    
    def __str__(self):
        """Genera una cadena descriptiva del objeto

        Returns
        -------
        str
            Cadena descriptiva
        """      
        return "Servidor: {}, país: {}, capacidad: {} GB.".format(self.identifier, self.country, self.capacity)
        
    def __repr__(self):
        """Genera una cadena descriptiva del objeto en colecciones

        Returns
        -------
        str
            Cadena descriptiva
        """      
        return "Servidor: {}".format(self.identifier)
    
            
    def rellena(self, videos):
        """Dada una colección de videos,
           seleccionar de cada uno cuanta cantidad (entre 0 y 1)
           se almacena en el servidor.
           Se ha de optimizar para que el tiempo de emisión
           sea el máximo posible.

        Parameters
        ----------
        videos : collection
            Colección de videos que se quieren almacenar en el servidor.
        """

        assert self.espacio_ocupado < self.capacity, "El servidor está lleno"
        videos_ordenados =  [] #Montículo

        for elem in videos:

            #Inserto el vídeo que más espectadores tenga:  (tam*esp/tam = esp)
            #Formato montículo (máximos): (-espectadores, Video) (guardo tuplas).
            heapq.heappush(videos_ordenados, (- elem.get_users(self.country), elem) )

        #En la primera iteración es 0
        cantidad_almacenada = self.espacio_ocupado

        #Mientras tenga hueco y queden vídeos
        while cantidad_almacenada < self.capacity and len(videos_ordenados) > 0:

            video_actual = heapq.heappop(videos_ordenados)[1]
            espacio_restante = self.capacity - cantidad_almacenada

            if espacio_restante > video_actual.size:
                proporcion = 1

            else:
                proporcion = espacio_restante / video_actual.size

            self.videos_almacenados[video_actual] = proporcion
            cantidad_almacenada += video_actual.size * proporcion

        #Se ha llenado el server (por si hay dos llamadas consecutivas)
        self.espacio_ocupado = cantidad_almacenada

    def disponible(self, video):
        """Obtiene la cantidad de vídeo disponible en el servidor.

        Parameters
        ----------
        video : Video object
            Vídeo del cual se quiere saber la disponibilidad

        Returns
        -------
        float
            Cantidad del vídeo disponible          
        """ 
        cantidad = self.videos_almacenados.get(video)

        if cantidad is None:
            return 0

        return cantidad

    def almacenados(self):
        """Material almacenado en el servidor

        Returns
        -------
        set
            Conjunto de tuplas (video, cantidad) de los videos ALMACENADOS en el servidor.          
        """ 
        return self.videos_almacenados.items()

    def tiempo_emision(self):
        """A partir de los datos almacenados
           devolver el tiempo de emisión
           siguiendo la fórmula: 
           \sum_{i}^{v} \text{espectadores}_i*\text{tamaño}_i*\text{porcionAlmacenada}_i

        Returns
        -------
        number
            Tiempo de emision disponible           
        """

        tiempo_emision = 0

        for tupla in self.videos_almacenados.items():
            tiempo_emision += ( tupla[0].size * tupla[0].get_users(self.country) * tupla[1] )

        return tiempo_emision


    #Metodos propios

    """Compara dos objetos.

        Returns
        -------
        bool
            True si el primero es menor que el segundo, false si no.
    """

    #Para el montículo (comparar "2" servidores en la U-P)
    def __lt__(self, other):
        return self.identifier < other.identifier


In [5]:
class ServidorMaestro:
    """
    Servidor central que gestiona las conexiones entre servidores cache
    """
    
    def __init__(self, servidores, distancias):
        """Instancia el servidor central

        Parameters
        ----------
        servidores : Iterable
            Conjunto de servidores cache disponibles
        distancias : dict{ServidorCache: dict{ServidorCache: int}}
            Grafo de distancias en milisegundos entre servidores.

            Diccionario diccionario, los servidores son hasheables
        """
        self.servidores = set(servidores)
        self.distancias = distancias

        #Grafo simplificado. Mismo formato que el no simplificado.
        self.simplificado = {}
    
    def get_grafo(self):
        """Devuelve el grafo de distancias recibido

        Returns
        -------
        dict{ServidorCache: dict{ServidorCache: int}}
            Grafo de distancias en milisegundos entre servidores.
        """
        return self.distancias

    def get_grafo_simplificado(self):
        """Devuelve el grafo de distancias simplificado

        Returns
        -------
        dict{ServidorCache: dict{ServidorCache: int}}
            Grafo de distancias en milisegundos entre servidores.
        """
        assert len(self.simplificado) > 0, "El grafo no ha sido simplificado todavía."
        return self.simplificado

    def simplifica_grafo(self):
        """A partir del grafo de distancias
           hacer una simplificación de la estrucutra
           de datos para ahorrar espacio y tiempo.
        """

        #METER MONTÍCULO
        pares_nodos_enlace = set() #Para no repetir enlaces en el montículo
        aristas = [] #Montículo ordenado

        for servidor_pings in self.distancias.items():

            for enlace in servidor_pings[1].items():

                #Comprobación para no meter enlaces repetidos al montículo
                if (servidor_pings[0], enlace[0]) not in pares_nodos_enlace and (enlace[0], servidor_pings[0]) not in pares_nodos_enlace:

                    #Montículo: ordeno según la distancia (si coinciden según el id) -> guardo distancia, server 1, server 2
                    heapq.heappush(aristas, (enlace[1], servidor_pings[0], enlace[0]))
                    pares_nodos_enlace.add((servidor_pings[0], enlace[0]))

        #KRUSKAL
        estructura_up = UnionPertenencia(self.servidores)
        numero_enlaces = 0

        #Salgo cuando todos los nodos están unidos o cuando no quedan enlaces
        #Todos los (n) nodos están unidos si hay (n-1) enlaces

        while len(aristas) != 0 and numero_enlaces != (len(self.servidores) - 1):

            arista = heapq.heappop(aristas)
            clase_nodo_uno = estructura_up.buscar(arista[1])
            clase_nodo_dos = estructura_up.buscar(arista[2])

            if clase_nodo_uno != clase_nodo_dos:

                estructura_up.unir(clase_nodo_uno, clase_nodo_dos)
                numero_enlaces += 1

                #Ahora los añadimos al diccionario resultado el enlace.
                self.anadir_diccionario_simplificado(arista[1], arista[2], arista[0])
                self.anadir_diccionario_simplificado(arista[2], arista[1], arista[0])

    def anadir_diccionario_simplificado(self, nodo_uno, nodo_dos, ping):

        """Añade los elementos al diccionario simplificado
        en el mismo formato que el no simplificado.

        Parameters
        ----------
        nodo_uno : nodo externo
        nodo_dos : nodo interno
        ping : latencia

        """

        #Ahora los añadimos al diccionario resultado (mismo formato que el no simplificado)
        diccionario = self.simplificado.get(nodo_uno)

        if diccionario is None:
            diccionario = {}

        diccionario[nodo_dos] = ping
        self.simplificado[nodo_uno] = diccionario

    def mas_cercano(self, servidor):

        """Reporta el servidor más cercano al dado por parámetro

        Parameters
        ----------
        servidor : ServidorCache

        Returns
        -------
        ServidorCache
            Servidor más cercano
        """

        if len(self.simplificado) > 0:
            grafo = self.simplificado

        else:
            grafo = self.distancias

        ping_minimo = 1000000
        tupla_minima = None

        for tupla in grafo[servidor].items():

            if tupla[1] < ping_minimo:
                ping_minimo = tupla[1]
                tupla_minima = tupla

        assert tupla_minima is not None, "Los nodos no están relacionados"
        return tupla_minima[0]

In [6]:
class UnionPertenencia:

    """
    Clase que proporciona una estructura unión pertenencia.
    """

    def __init__(self, nodos):

        """Constructor

        Parameters
        ----------
        nodos : Conjunto de nodos del árbol
        """

        self.estructura = {}

        # Guardardo para cada nodo su Padre y la Altura
        for nodo in nodos:
            self.estructura[nodo] = (nodo, 0)

    def buscar (self, nodo):

        """Devuelve la clase de un nodo dado.
        Limitada por la altura -> Búsqueda de como mucho lg(nodos)

        Parameters
        ----------
         nodo : nodo para buscar su clase.

        Returns
        -------
        nodo : clase del nodo inicial.
        """

        padre_nodo = self.estructura.get(nodo)[0]

        while padre_nodo != nodo:

            nodo = padre_nodo
            padre_nodo = self.estructura.get(nodo)[0]

        return nodo


    def unir (self, clase_uno, clase_dos):

        """Se realiza la unión de dos clases dadas.
        Esta unión está limitada por la altura.

        Parameters
        ----------
         clase_uno : clase a unir
         clase_dos : clase a unir
        """

        altura_uno = self.estructura.get(clase_uno)[1]
        altura_dos = self.estructura.get(clase_dos)[1]

        #Si las alturas son iguales
        if altura_uno == altura_dos:

            self.estructura[clase_uno] = (self.estructura[clase_uno][0], altura_uno + 1)
            self.estructura[clase_dos] = (clase_uno, altura_dos)

        elif altura_uno > altura_dos:
            self.estructura[clase_dos] = (clase_uno, altura_dos)

        else:
            self.estructura[clase_uno] = (clase_dos, altura_uno)

### Pruebas de ejemplo

In [7]:
import unittest
import json

def carga_dataset(data):
    with open(data) as f:    
        test_datasets = json.load(f)

    videos = list()
    for v in test_datasets["videos"]:
        v_obj = Video(v["name"], v["size"])
        for c, u in v["users"].items():
            v_obj.set_users(c, u)
        videos.append(v_obj)
            
    servers = dict()
    for s in test_datasets["servers"]:
        servers[s["country"]] = ServidorCache(s["identifier"], s["country"], s["size"])

    
    pings = test_datasets["pings"]
    p_ = dict()
    for s in servers.values():
        p_[s] = dict()
        for p in pings[s.country]:
            p_[s][servers[p]] = pings[s.country][p]
    maestro = ServidorMaestro(servers.values(), p_)
    
    return videos, servers, maestro

class TestCargaServidores(unittest.TestCase):
    
    def test_carga_simple(self):
        
        v, s, m = carga_dataset("toy.json")

        spain = s["Spain"]
        spain.rellena(v)
        self.assertEqual(spain.tiempo_emision(), 578000)

        #spain.rellena(v) #Salta el asserto

        almacenados = spain.almacenados()
        self.assertIn((v[3], 0.5), almacenados)
        self.assertEqual(v[3].get_users("Spain"), 400)
        
        m.simplifica_grafo()
        self.assertEqual(m.mas_cercano(s["Spain"]), s["France"])

        m.simplifica_grafo()
        self.assertEqual(m.mas_cercano(s["Spain"]), s["France"])
        self.assertEqual(m.mas_cercano(s["France"]), s["Spain"])

        #Aquí hay más tests míos. No los elimino debido a que se comentó en clase que se valorarían positivamente.

        #Resultado tras aplicar Kruskal
        # resultado = {s['Spain']: {s['France']: 50}, s['France']: {s['Spain']: 50, s['Ireland']: 100, s['Poland']: 250}, s['Ireland']: {s['France']: 100}, s['Poland']: {s['France']: 250}}
        # self.assertEqual(m.simplificado, resultado)
        #
        #
        # #Más pruebas -> otro fichero
        #
        # v2, s2, m2 = carga_dataset("toy_mio.json")
        #
        # #Solo sirve si se cambia el tamaño de Irlanda a 1000
        # ireland = s2["Ireland"]
        # ireland.rellena(v2)
        # self.assertEqual(ireland.tiempo_emision(), 960000)
        # self.assertIn((v2[4], 0.6), ireland.almacenados())
        #
        # self.assertEqual(v[3].get_users("Japan"), 0)
        #
        # m2.simplifica_grafo()
        # self.assertEqual(m2.mas_cercano(s2["Spain"]), s2["Japan"])
        # self.assertEqual(m2.mas_cercano(s2["Spain"]), s2["Japan"])
        # self.assertEqual(m2.mas_cercano(s2["France"]), s2["Spain"])
        # self.assertEqual(m2.mas_cercano(s2["Korea"]), s2["China"])
        # self.assertEqual(m2.mas_cercano(s2["Poland"]), s2["Spain"])

In [8]:
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK
